In [185]:
using DataFrames, CSV, Measurements, Unitful, Statistics
UNITFUL_FANCY_EXPONENTS = false # helps display better on graphs
using Interpolations
using Plots
using LaTeXStrings
using StringEncodings
using UnitfulLatexify
using MathTeXEngine
import LinearAlgebra.dot as ⋅
# https://github.com/JuliaPhysics/PhysicalConstants.jl

In [2]:
scalefontsizes(1.2) # Makes all fonts larger. 
# ^ Repeatedly calling this keeps scaling the font, so we set it aside.

In [17]:
Plots.gr()
#Plots.plotlyjs()
#Plots.pythonplot() # Important for working with unitful units
default(fontfamily = "Helvetica", #"Computer Modern", # Serif,
        framestyle = :box,
        grid = false,
        label = nothing, # Disables the default label that appears.
        linewidth = 2,
        #ticks = :native, # Allows interactive zooming with appropriate tick marks.
        #size = 1500 .* (3/2, 1),
        palette = :Paired_10, # https://docs.juliaplots.org/latest/generated/colorschemes/
        title_align = :left,
        titlefontvalign = :top,
        legendposition = :topright,
        size = [920, 690],
        margin = (10, :px),
        )
# https://discourse.julialang.org/t/nice-fonts-with-plots-gr-and-latexstrings/60037/4

# The Scheme.

1. Thermal distribution of velocities of a Yb gas in our cell.

2. Doppler shift of pump frequency given velocity.

3. Distribution of detuning due to doppler shift from thermal velocity.

# Other references.

* [The Yb Hollow Cathode Lamp (HCL).](https://www.perkinelmer.com/product/lumina-hollow-cathode-2-lamp-yb-n3050190)

In [12]:
# Constants.
c = u"c0";
m_Yb = (173.04 ± 0.01)u"u"; # Atomic mass of Yb in atomic mass units. https://physics.nist.gov/PhysRefData/Handbook/Tables/ytterbiumtable1.htm.
T_boiling_Yb = 1469u"K"; # Boiling point of Yb.
ω_laser = u"Hz"(2π * u"c0" / (399 ± 1)u"nm");
ω_width_laser = 2π * 10u"kHz" # https://www.toptica.com/products/tunable-diode-lasers/ecdl-dfb-lasers/dl-pro/

In [146]:
# Maxwell-Boltzmann distribution of thermal velocities for a classical gas. Produces a unitless probability per speed.
# v = velocity in question.
# m = particle mass.
# T = temperature of entire gas (must be uniform, in equilibrium, etc.)
# u"k" = Unitful package constant, the Boltzmann constant.
function MB_pdf(v, m, T) 
    return u"(m/s)^-3"((m / (2π * u"k" * T))^(3/2) * exp(-m * (sum([x^2 for x ∈ v])) / (2 * u"k" * T)))
end
MB_max(m, T) = MB_pdf(sqrt(2u"k" * T / m) * [1, 0, 0], m, T)

MB_max (generic function with 1 method)

In [166]:
# Return a vector pulled from the given distribution.
# distribution = a function which takes 1 vector and returns its normalized probability.
# bounds = N pairs in a list, where N is the dim of the vector. Each pair specifies a rectangular bound along one axis.
# N = vector dimension.
# Distribution_max = peak value of the PDF.
function generate_vector(distribution, distribution_max, bounds)
    N = length(bounds) # Vector dimensions.
    monte_carlo_success = false
    v = zeros(N) * u"m/s"
    while monte_carlo_success == false # Loop stops when the machine wins the MC coinflip.
        r = rand(Float64, N)
        # Put it into the bounds
        for i ∈ 1:N
            lower = bounds[i][1]
            upper = bounds[i][2]
            length = upper - lower
            v[i] = (r[i] * length + lower) # Scales to size of that time, then moves it to the bottom
        end
        # Flip the MC coin.
        monte_carlo_success = (rand(Float64) * distribution_max <= distribution(v))
    end
    return v
end

generate_vector (generic function with 2 methods)

In [175]:
thermal_distribution = (v -> MB_pdf(v, m_Yb, T_boiling_Yb))
bounds = [[-1e3u"m/s", 1e3u"m/s"], [-1e3u"m/s", 1e3u"m/s"], [-1e3u"m/s", 1e3u"m/s"]]

3-element Vector{Vector{Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(m, s⁻¹), 𝐋 𝐓⁻¹, nothing}}}}:
 [-1000.0 m s⁻¹, 1000.0 m s⁻¹]
 [-1000.0 m s⁻¹, 1000.0 m s⁻¹]
 [-1000.0 m s⁻¹, 1000.0 m s⁻¹]

In [182]:
N_thrown = 1000
velocities = [generate_vector(thermal_distribution, MB_max(m_Yb, T_boiling_Yb), bounds) for x ∈ 1:N_thrown]
Plots.plotlyjs()
scatter3d(getindex.(velocities, 1), getindex.(velocities, 2), getindex.(velocities, 3),
    title = "Distribution of Yb gas velocities (T = $T_boiling_Yb, N = $N_thrown)",)

In [202]:
histogram([sqrt(v ⋅ v) for v ∈ velocities],
    title = "Speed distribution for Yb gas (T = $T_boiling_Yb, N = $N_thrown)",
    xlabel = "Speed",
    ylabel = "Number of atoms")

In [203]:
# Increase in ω_0 due velocity v.
# v = velocity vector, measured in the w_0 source frame.
function doppler_shift(v, ω_0, source_k̂) 
    γ = 1 / sqrt(1 - (v ⋅ v) / c^2)
    return ω_0 * (γ * (1 - ((v/c) ⋅ source_k̂)) - 1)
end
#doppler_shift(v, ω_0) = ω_0 * (sqrt((u"c0" + v) / (u"c0" - v)) - 1)

# Gives the speed |v| as a function of the doppler shift.
# Inverse of doppler_shift
#speed(s, ω_0) = (-c * s^2 - 2 * c * s * ω_0) / (s^2 + 2s * ω_0 + 2ω_0^2)

speed (generic function with 1 method)

In [204]:
ω_laser = u"THz"(2π*u"c0" / (399 ± 1)u"nm")
# Assumed direction of the pump light.
pump_k̂ = [0, 0, 1];

In [205]:
doppler_shift([0, 0, -1e3]u"m/s", ω_laser, pump_k̂)

0.015747 ± 3.9e-5 THz

In [227]:
vs = range(-2.5e6, 2.5e6, length = 100)u"m/s"
doppler_shift(10u"m/s", ω_laser)

N_thrown = 1000
velocities = [generate_vector(thermal_distribution, MB_max(m_Yb, T_boiling_Yb), bounds) for x ∈ 1:N_thrown]
shifts = [doppler_shift(v, ω_laser, pump_k̂) for v ∈ velocities]
histogram(shifts,
    #xscale = :log10,
    xlims = (0u"MHz", 0.02u"THz"),
    #xunit = u"MHz",
    ylims = (0, :auto))

#=
plot(vs, doppler_shift.(vs, ω_laser) .* (1 / 2π),
    title = "1D Laser detuning (ω_0 = $ω_laser)",
    #ylabel = "Frequency change",
    ylabel = L"\Delta\omega",
    xlabel = "Velocity",
    yunit = u"THz")
=#

In [14]:
# Derivative of speed with respect to doppler shift s.
∂v_∂s(s, ω_0) = (4c * ω_0^2 * (s + ω_0)) / (s^2 + 2s * ω_0 + 2 * ω_0^2)^2

∂v_∂s (generic function with 1 method)

In [231]:
# Distribution of doppler shifts (detuning) for a classical gas at temperature T, mass m.
# Use absolute value because PDFs should be strictly > 0.
#shift_pdf(s, ω_0, m, T) = u"Hz^-1"(MB_pdf(speed(s, ω_0), m, T) * abs(∂v_∂s(s, ω_0)))

shift_pdf(s, ω_0, m, T) = ((2π * c) / ω_0 * (sqrt(32) * pi)^-1 * exp(-c^2 * m * s^2 / (u"k" * T * ω_0^2)))

shift_pdf (generic function with 1 method)

In [229]:
linewidth_399_transition = 20u"MHz" # Page 87 on red notebook.;

20 MHz

In [236]:
v_effective_support = range(-0.5e3, 0.5e3, length = 100)u"m/s"
s_eff_support = doppler_shift.(v_effective_support, ω_laser)
ω_to_λ(ω) = 2π*c/ω
T = T_boiling_Yb
pdf_values = shift_pdf.(s_eff_support, ω_laser, m_Yb, T)
plot(s_eff_support ./ 2π, pdf_values,
    title = "Rough doppler-broadening in 1D Yb gas (T = $T)",
    ylabel = "PDF",
    xunit = u"MHz",
    #xscale = :log10,
    ylims = (0, :auto),
    xlabel = "Wavelength detuning")
plot!([linewidth_399_transition, linewidth_399_transition], [minimum(pdf_values), maximum(pdf_values)],
    linestyle = :dash,
    linewidth = 2,
    label = "1S0 → 1P1 transition linewidth",
    color = :orange)

In [42]:
u"MHz"(c/399u"nm")

7.513595438596491e8 MHz

# Population simulation.
1. Draw initial velocities for $N$ atoms.
2. Set each atom's state to ground.
3. Calculate detunings for each atom.
4. Advance by one timestep.
5. Advance each atom's $c_2$ coefficient according to the time passed and the atom's detuning.
6. Roll for spontaneous and stimulated processes for each atom. Set atom to ground to excited as necessary.
7. Find the average $c_2^2$ value for the population.
8. Repeat for the desired number of timesteps and plot average $c_2^2$ over time.

# Steady-state plot.